<a href="https://colab.research.google.com/github/aSafarpoor/KGCLG/blob/main/LightGCN_part1_view0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tqdm import tqdm 
import pandas as pd
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/KGCLG

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/KGCLG


In [4]:
!ls 

brands1.pickle			     meta_justV.pickle
catedges1.pickle		     meta_justV.rar
iinits.pickle			     meta.pickle
Industrial_and_Scientific.json	     nodeembeddings.json
meta128_2.csv			     uinits.pickle
meta128.pickle			     ui.pickle
meta_Industrial_and_Scientific.json  view0_nodeembeddings.json
meta_justV			     view0.pickle


In [5]:
# with open('ui.pickle', 'wb') as handle:
    # pickle.dump(ui, handle)
def pickle_reader(name):
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
        return b

In [6]:
'''
#should to be read from hard drive
* brands1.pickle			     
* catedges1.pickle
* meta128.pickle
* ui.pickle
'''
brandedges = pickle_reader("brands1.pickle")
catedges = pickle_reader("catedges1.pickle")
meta = pickle_reader("meta128.pickle")
ui = pickle_reader("ui.pickle")

In [7]:
# meta.to_csv('meta128_2.csv')

In [ ]:
ne = {}
users = list(set(list(ui.reviewerID)))
items = list(set(list(ui.asin)))
for i in tqdm(items):
    ne[i] = np.array(meta[meta.asin==i].text2vec)
for i in tqdm(users):
        ne[i] = np.zeros(128) 

 93%|█████████▎| 122238/131400 [17:35<01:16, 119.12it/s]

In [ ]:
len(users),len(items)

In [ ]:
nvd = {}
cd = {}

for n in tqdm(ne):
    nvd[n]=[]
    cd[n]=0


In [ ]:
# create_node_vicinity from ui:
for ind in tqdm(range(len(ui))):
    row = ui.iloc[ind]
    e1, e2 = row['asin'],row['reviewerID']
    cd[e2] += 1    
    nvd[e2].append(e1)

In [12]:
for u in ne:
    if len(ne[u])==1:
        ne[u]=ne[u][0]

for u in tqdm(nvd):
    vs = nvd[u];
    if len(vs)>0:
        for v in vs:
             ne[u] += ne[v]
        ne[u] /= cd[u] 
    else:
        pass

100%|██████████| 1045550/1045550 [05:31<00:00, 3151.79it/s]


In [13]:
type(ne)

dict

In [14]:
ne[v]

0     -3.393732
1     -2.733804
2      1.432690
3     -1.666343
4      0.404424
         ...   
123   -0.093065
124    0.176109
125   -0.080944
126    0.041555
127    0.135397
Length: 128, dtype: float32

In [15]:
ne[u]

0     -3.393732
1     -2.733804
2      1.432690
3     -1.666343
4      0.404424
         ...   
123   -0.093065
124    0.176109
125   -0.080944
126    0.041555
127    0.135397
Length: 128, dtype: float64

In [19]:
type(ne[v]),type(ne[u])

(pandas.core.series.Series, pandas.core.series.Series)

In [21]:
d2 = [[],[]]

In [22]:
for i in ne:
    d2[0].append(i)
    d2[1].append(ne[i])    


In [27]:
d3 = np.array(d2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [28]:
d3.shape

(2, 1045550)

In [29]:
d4 = d3.T

In [32]:
d4.shape

(1045550, 2)

In [33]:
df = pd.DataFrame (d4, columns = ['name','vector'])

In [34]:
df

,name,vector
0,B003ULPA9M,0 -2.751240 1 1.963953 2 -2.37910...
1,B00PZYIEEU,0 5.407245 1 -3.444118 2 2.32683...
2,B01BXGCYBM,0 -4.327692 1 1.152166 2 -0.25391...
3,B00BQM6XKW,0 -5.215747 1 2.388364 2 0.60712...
4,B00NHRNFBO,0 -5.553534 1 -1.249214 2 -3.42922...
...,...,...
1045545,A1UZNJ0W78BU0,0 3.091146 1 0.147855 2 0.66450...
1045546,A1NE7HMQ7W4W0P,0 -5.994321 1 3.724970 2 3.06819...
1045547,A36UNCDNUVDWH5,0 2.915727 1 3.518314 2 -3.14140...
1045548,A2GE7LL651OCSV,0 -0.960481 1 3.048429 2 4.84720...


In [35]:
df.to_csv('view1-embeddings.csv')

In [38]:
!mv view1-embeddings.csv view0-embeddings.csv

In [39]:
nvd = {}

for n in tqdm(users):
    nvd[n]=[]
for n in tqdm(items):
    nvd[n]=[]

100%|██████████| 131400/131400 [00:04<00:00, 28962.39it/s]


In [40]:
for ind in tqdm(range(len(ui))):
    row = ui.iloc[ind]
    e1, e2 = row['asin'],row['reviewerID']
    nvd[e1].append(e2)   
    nvd[e2].append(e1)

100%|██████████| 1267323/1267323 [01:19<00:00, 15900.92it/s]


In [45]:
import csv

w = csv.writer(open("view0_edges_dict.csv", "w"))

for key, val in nvd.items():
    w.writerow([key, val])